In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install streamlit

In [ ]:
!pip install streamlit pyngrok

In [ ]:
%%writefile streamlitapp.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import joblib

# Page config
st.set_page_config(page_title="Credit Card Fraud Detection", page_icon="💳", layout="wide")

# Title
st.title("💳 Credit Card Fraud Detection ")
st.write("Interactive Credit Card Fraud Detection and Analysis using Logistic Regression with Streamlit")

# Load Dataset
data_path = "/content/Final Transactions.csv"
df = pd.read_csv(data_path)

# Sidebar: sample size control
sample_size = st.sidebar.slider(
    "Select sample size for visualizations",
    min_value=100, max_value=500, value=500, step=100
)

# Use the selected sample size
df_sample = df.sample(sample_size, random_state=42)

#Data section
st.subheader("📋Data Preview")
st.write("Here are the first 10 rows of the dataset:")
st.dataframe(df.head(10))

# Show basic info (shape)
st.write(f"Dataset has **{df.shape[0]} rows** and **{df.shape[1]} columns**.")

# Show summary stats
st.subheader("📑Statistical Summary")
st.write(df.describe())

# EDA Section
st.subheader("📊Exploratory Data Analysis")

# Count Plot
st.subheader("Fraud vs Non-Fraud Count")
fig, ax = plt.subplots(figsize=(6,4))
sns.countplot(data=df_sample, x="TX_FRAUD", palette="Set2", ax=ax)
ax.set_title(f"Fraud vs Non-Fraud Count (Sampled {sample_size})")
ax.set_xlabel("Non-Fraud(0) vs Fraud (1)")
ax.set_ylabel("Count")
st.pyplot(fig)

# Histogram
st.subheader("Transaction Amount Distribution")
fig, ax = plt.subplots(figsize=(6,4))
sns.histplot(data=df_sample, x="TX_AMOUNT", hue="TX_FRAUD", bins=30, kde=False, ax=ax)
ax.set_title("Transaction Amount Distribution by Fraud Status")
st.pyplot(fig)

# Scatter plot
st.subheader("📈Logistic Regression Curve")
from sklearn.linear_model import LogisticRegression

if "TX_FRAUD_SCENARIO" in df.columns and "TX_FRAUD" in df.columns:
    X = df[['TX_FRAUD_SCENARIO']]
    y = df['TX_FRAUD']

    # Fit logistic regression
    log_reg = LogisticRegression()
    log_reg.fit(X, y)

    # Generate smooth X for S-curve
    x_range = np.linspace(X.min(), X.max(), 300).reshape(-1, 1)
    y_prob = log_reg.predict_proba(x_range)[:, 1]

    # Plot
    fig, ax = plt.subplots(figsize=(8, 5))
    # Scatter with colors based on class
    ax.scatter(X[y==0], y[y==0], color="blue", alpha=0.6, label="Non-Fraud (0)")
    ax.scatter(X[y==1], y[y==1], color="red", alpha=0.6, label="Fraud (1)")

    # Logistic curve
    ax.plot(x_range, y_prob, color="black", linewidth=2, label="Logistic S-Curve")

    ax.set_xlabel("TX_FRAUD_SCENARIO")
    ax.set_ylabel("TX_FRAUD")
    ax.set_title("Logistic Regression Curve (Blue=Non-Fraud, Red=Fraud)")
    ax.legend()
    st.pyplot(fig)
else:
    st.warning("⚠️ Columns 'TX_FRAUD_SCENARIO' and 'TX_FRAUD' are not available in the dataset.")

# Load Pre-trained Model
model = joblib.load("/content/creditcardmodel.pkl")

# --- Prediction Section ---
st.subheader("🎛️Test a Transaction")

fraud_scenario = st.text_input(
    "Enter Fraud Scenario Number:",
    value="0"
)

if st.button("🔍 Predict"):
    try:
        # Convert input string to integer
        fraud_scenario = int(fraud_scenario.strip())
        input_data = [[fraud_scenario]]  # numeric 2D array

        prediction = model.predict(input_data)[0]

        if prediction == 1:
            st.error("⚠️ This transaction is predicted as FRAUD!")
        else:
            st.success("✅ This transaction is predicted as GENUINE.")

    except ValueError:
        st.warning("⚠️ Please enter a valid numeric value.")

# Footer
st.markdown("---")
st.markdown("Made with ❤️ using Streamlit")


Writing streamlitapp.py


In [ ]:
# for Colab Version only
!curl ipv4.icanhazip.com

35.203.156.80


In [ ]:
import subprocess
subprocess.Popen(["streamlit", "run", "/content/streamlitapp.py", "--server.port", "8501", "--server.address", "0.0.0.0"])

<Popen: returncode: None args: ['streamlit', 'run', '/content/streamlitapp.p...>

In [ ]:
!ngrok config add-authtoken "31RhHVj0F9FITuE7AaDdxwnQm8Q_3rr1p6i9Q9ViQdseaYgGc"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print("🌍 Public URL:", public_url)

🌍 Public URL: NgrokTunnel: "https://4fc45916a32b.ngrok-free.app" -> "http://localhost:8501"
